# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [4]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['employees', 'orders', 'products'],  # Tabels name
    'definition': [
        'This table contains employee details like employee_id, name, and position.',
        'This table contains order information including order_id, customer_id, and order_date.',
        'This table contains product details such as product_id, name, and price.'
    ]
}

df = pd.DataFrame(data)
df

,table,definition
0,employees,This table contains employee details like empl...
1,orders,This table contains order information includin...
2,products,This table contains product details such as pr...


In [5]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [6]:
print(text_tables)

employees: This table contains employee details like employee_id, name, and position.
orders: This table contains order information including order_id, customer_id, and order_date.
products: This table contains product details such as product_id, name, and price.


In [7]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
{question}
"""


In [8]:
#Creating the prompt, with the user questions and the tables definitions.                                   
pqt1 = prompt_question_tables.format(tables=text_tables, question="I need to find the orders placed by employees with a certain position.")
                                    

In [9]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "orders"]
}


In [10]:
question = "What is the total number of orders placed by each employee?"

# استبدال القيم داخل القالب
pqt3 = prompt_question_tables.format(tables=text_tables, question=question)

# طباعة النتيجة
print(pqt3)


Given the following tables and their content definitions,
###Tables
employees: This table contains employee details like employee_id, name, and position.
orders: This table contains order information including order_id, customer_id, and order_date.
products: This table contains product details such as product_id, name, and price.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Question:
What is the total number of orders placed by each employee?



In [11]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "orders"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [21]:
import os
import openai
import pandas as pd
from dotenv import load_dotenv

# تحميل متغيرات البيئة
load_dotenv()

# تعريف API Key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# التحقق من أن API Key تم تحميله بشكل صحيح
if OPENAI_API_KEY is None:
    raise ValueError("⚠️ Error: OPENAI_API_KEY is not set. Please check your .env file.")

# تهيئة العميل
client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [22]:
def query_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
    )
    return response.choices[0].message.content


In [23]:
# بيانات الموظفين والطلبات (كمثال)
data = {
    "Employee": ["Alice", "Bob", "Charlie", "David"],
    "Orders": [120, 85, 98, 150],
    "Department": ["Sales", "Marketing", "Sales", "Tech"]
}

df = pd.DataFrame(data)
print(df)


  Employee  Orders Department
0    Alice     120      Sales
1      Bob      85  Marketing
2  Charlie      98      Sales
3    David     150       Tech


In [32]:
prompt1 = """
What is the total number of orders placed by each employee?
"""

response1 = query_gpt(prompt1)
print(response1)


To determine the total number of orders placed by each employee, you would need a dataset that includes information on the orders placed and the employees responsible for placing them. You would then need to group the orders by employee and count the number of orders placed by each employee. This can be done using SQL queries or spreadsheet functions such as COUNTIF.


In [33]:
prompt2 = """
Show me the top 5 best-selling products along with their total sales amount and percentage of overall revenue.
"""

response2 = query_gpt(prompt2)
print(response2)


1. Product A - Total sales amount: $100,000 (20% of overall revenue)
2. Product B - Total sales amount: $80,000 (16% of overall revenue)
3. Product C - Total sales amount: $70,000 (14% of overall revenue)
4. Product D - Total sales amount: $60,000 (12% of overall revenue)
5. Product E - Total sales amount: $50,000 (10% of overall revenue)


In [34]:
prompt3 = """
I’m exploring regional market trends. Can you provide a heatmap-style summary of orders per region, and suggest where we should focus our marketing efforts?
"""

response3 = query_gpt(prompt3)
print(response3)


Sure! Here is a heatmap-style summary of orders per region:

- Northeast: High order volume
- Southeast: Moderate order volume
- Midwest: Low order volume
- West: Moderate order volume

Based on this data, it seems like the Northeast region has the highest demand for our products. Therefore, it would be beneficial to focus our marketing efforts in that region to capitalize on the high order volume. Additionally, we may want to consider increasing our marketing efforts in the West region, as they have a moderate order volume and potential for growth.
